#Op Return Layer 2

<a href="https://colab.research.google.com/github/ProfDoeg/Colegio_Invisible/blob/master/11_cuaderno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparar el Espacio (Prepare the Space)

In [ ]:
! pip install eciespy eth_keys cryptos qrcode base58 
! git clone https://github.com/ProfDoeg/Colegio_Invisible.git

# Importaciones y Definiciones (Imports and Definitions)

Crear funciones para importar claves y direcciones de criptomonedas.
***
Create functions for importing keys and cryptocurrency addresses.

In [ ]:
import pandas as pd
from cryptos.py3specials import safe_hexlify, from_string_to_bytes, from_int_to_byte, from_string_to_bytes
from cryptos import serialize,deserialize

import hashlib
import os
import getpass
import ecies
import eth_keys
import cryptos
import qrcode
import base58
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def import_privKey(privkey_filepath,password=None):
    if password==None:
        password = getpass.getpass("Input password for decrypting keyfile: ")
    password=password.encode()
    f=open(privkey_filepath,'rb')
    decrypted_bytes=ecies.aes_decrypt(key=hashlib.sha256(password).digest(),cipher_text=f.read())
    privKey=eth_keys.keys.PrivateKey(decrypted_bytes)
    return privKey

def import_pubKey(pubkey_filepath):
    f=open(pubkey_filepath,'rb')
    privKey=eth_keys.keys.PublicKey(f.read())
    return privKey

def import_addr(addr_filepath):
    return open(addr_filepath,'rb').read().decode()

def only_conf(utxos):
    return [utxo for utxo in utxos
            if doge.fetchtx(utxo['output'].split(':')[0])['confirmations']>0]

def mk_opreturn(msg, rawtx=None, json=0):
    
    def op_push(data):
        if type(data) == bytes:
            bytedata=data
        else:
            bytedata=data.encode()
        import struct
        if len(data) < 0x4c:
            return from_int_to_byte(len(bytedata)) + bytedata
        elif len(data) < 0xff:
            return from_int_to_byte(76) + struct.pack('<B', len(bytedata)) + bytedata
        elif len(data) < 0xffff:
            return from_int_to_byte(77) + struct.pack('<H', len(bytedata)) + bytedata
        elif len(data) < 0xffffffff:
            return from_int_to_byte(78) + struct.pack('<I', len(bytedata)) + bytedata
        else: raise Exception("Input data error. Rawtx must be hex chars" \
                            + "0xffffffff > len(data) > 0")

    orhex = safe_hexlify(b'\x6a' + op_push(msg))
    orjson = {'script' : orhex, 'value' : 0}
    if rawtx is not None:
        try:
            txo = deserialize(rawtx)
            if not 'outs' in txo.keys(): raise Exception("OP_Return cannot be the sole output!")
            txo['outs'].append(orjson)
            newrawtx = serialize(txo)
            return newrawtx
        except Exception as E:
            print(E)
            print(type(E))
            raise Exception("Raw Tx Error!")
    return orhex if not json else orjson

# Comenzar con Claves Existentes (Start with Existing Keys)

Acceder a las claves generadas en el cuaderno 5
***
Access the keys generated in the cuaderno 5

## Comenzar con Claves en Google Drive (Start with Keys on Google Drive)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! cp drive/MyDrive/cinv/llaves.zip llaves.zip 
! unzip llaves.zip


## Comenzar con Claves en la Disco Local
## (Start with Keys on Local Drive)

In [ ]:
#from google.colab import files
#fu=files.upload();

In [ ]:
#! unzip llaves.zip

In [ ]:
#! ls llaves

# Import Private Key, Public Key and Address

In [ ]:
from IPython.display import Image
qr=Image('llaves/mi_addr.png')

privKey=import_privKey('llaves/mi_prv.enc','')
pub=import_pubKey('llaves/mi_pub.bin').to_hex()
addr=import_addr('llaves/mi_addr.bin')

display(qr)
display(pub)
display(addr)

In [ ]:
message='''Radio, live transmission.
Radio, live transmission.

Listen to the silence, let it ring on.
Eyes, dark grey lenses frightened of the sun.
We would have a fine time living in the night,
Left to blind destruction,
Waiting for our sight.

And we would go on as though nothing was wrong.
And hide from these days we remained all alone.
Staying in the same place, just staying out the time.
Touching from a distance,
Further all the time.

Dance, dance, dance, dance, dance, to the radio.
Dance, dance, dance, dance, dance, to the radio...

Well I could call out when the going gets tough.
The things that we've learnt are no longer enough.
No language, just sound, that's all we need know,
To synchronise love to the beat of the show.

And we could dance.

Dance, dance, dance, dance, dance, to the radio.
Dance, dance, dance, dance, dance, to the radio...
'''.encode()

In [ ]:
message1='''Bajan

Tengo tiempo para saber
Si lo que sueño concluye en algo
No te apures ya más loco
Porque es entonces cuando las horas
Bajan, el día es vidrio sin sol
Bajan, la noche te oculta la voz
Y además vos querés sol
Despacio también podés hallar la luna
Viejo roble del camino, tus hojas siempre se agitan algo
Nena, nena, que bien te ves
Cuando en tus ojos no importa si las horas
Bajan, la noche es día sin sol
Bajan, el día que nunca acabó
Y además vos sos el sol
Despacio también podés ser la luna'''.encode()

In [ ]:
type(message)

In [ ]:
len(message1)

In [ ]:
list(range(0, len(message), 76))

In [ ]:
#message='hello to the world. this is a message to everyone who likes gravy. slow down. it is not healthy. it is too thick and sloppy'.encode()
n = 76 
chunks = [message[i:i+n] for i in range(0, len(message), n)]
chunks

In [ ]:
def pieces(bites,size):
    return [bites[i:i+size] for i in range(0, len(bites), size)]

In [ ]:
def prepend_index(L):
    return [ int(i).to_bytes(1, 'big')+int(len(L)-1).to_bytes(1, 'big')+chunk for i,chunk in enumerate(L)]

In [ ]:
def prepend_head(L,byte_head):
    return [ byte_head+chunk for chunk in L]

# byte head contains leading byte "\xdd" and byte to represent embed type
# a single line text will be embed type "\x00"
# a single line hash will be embed type "\x01"
# a multiline text will be embed type "\x02"
# a multiline ecies encrypted text will be embed type "\x03"

In [ ]:
def multiline_setup(mensaje):
    mensaje_partes=pieces(mensaje,76)
    indexed_mensaje_partes=prepend_index(mensaje_partes)
    return prepend_head(indexed_mensaje_partes,b'\xdd\x02')

In [ ]:
chunks=pieces(message,76)
chunks

In [ ]:
indexed_chunks=prepend_index(chunks)
indexed_chunks

In [ ]:
headed_chunks=prepend_head(indexed_chunks,b'\xdd\x02')
headed_chunks

In [ ]:
opreturn_list=multiline_setup(message)
opreturn_list

In [ ]:
opreturn_list1=multiline_setup(message1)
opreturn_list1

In [ ]:
def embed_message_list(L,privkeyhex,tip=2_00_000_000):
    import time
    txn_ids=[]
    doge=cryptos.Doge()
    addy=doge.privtoaddr(privkeyhex)
    doge_unspents=only_conf( doge.unspent(addy) )
    doge_unspent_total=sum([unspent['value'] for unspent in doge_unspents])
    if ( tip*len(L) + 3_00_000_000 ) > doge_unspent_total:
        raise Exception("Not enough coins available")
    # at this point we know we have enough coins...
    # now we will iterate through the list
    for op_return in L:
        print(op_return)
    # make sure to wait for utxos to confirm, if not wait 10 sec
        while True:
            doge_unspents=only_conf( doge.unspent(addy) )
            if doge_unspents != []:
              print('COINS UP')
              break
            time.sleep(20)
        doge_unspent_total=sum([unspent['value'] for unspent in doge_unspents]) 
        doge_gift=doge_unspent_total-tip
        doge_outputs=[ {'value':doge_gift , 'address': addy}]
        doge_tx=doge.mktx(doge_unspents,doge_outputs)
        doge_serial_tx=cryptos.serialize(doge_tx)
        doge_inscribed_serial_tx=mk_opreturn( op_return , doge_serial_tx)
        doge_inscribed_tx=cryptos.deserialize(doge_inscribed_serial_tx)
        doge_signed_inscribed_tx=doge.signall(doge_inscribed_tx,privkeyhex)
        broadcast_doge_signed_inscribed_tx=doge.pushtx(doge_signed_inscribed_tx)
        txn_ids.append(broadcast_doge_signed_inscribed_tx['data']['txid'])
    return txn_ids

In [ ]:
embed_message_list(opreturn_list1,privKey.to_hex()[2:],tip=2_00_000_000)

In [ ]:
doge=cryptos.Doge()

In [ ]:
H=doge.history('D6zKNnkupqRbkB9p5rwix8QiobQWJazjyX')


In [ ]:
lyric_txs=[ tx['txid'] for tx in H['data']['txs'][:13]][::-1]
#print(lyric_txs.pop(7))
lyric_txs

In [ ]:
lyric_txs1=['74de5e9bf0c292287e08c201d26676e26a5c2f21ddb639e9268af6d52d0e0c1c',
 'd5e4be06e1b631a28d2bbea11b8e2ba0bbcae0c0a9bb44041fca368f65782b84',
 '50b6d86e343b7f34e28ca4ba09ec80da4b1c3a1d541a0c6718290d134301c5ef',
 'd163ae8b500eb3d5b32112b8fe60a8c089f424c4649afd3f8d29bf99fe2f4280',
 'bff8132fe5a0629cb2ea364c476c588da6fc0a2cd6d25738ae905082894de4b3',
 '352cc874e23088f47bf7d5bc195063dc30b62e88acb3ffcb2d97fc48cd704c60',
 '5eb370b059634ac152c1abf834f869dd7937e43f7c21ef6418022d3551f4766f']

In [ ]:
[ output['script'] for output in doge.fetchtx(lyric_txs[0])['outputs'] if 'OP_RETURN' in output['script']][0][10:]

In [ ]:
def tx2opreturn(txn):
    doge=cryptos.Doge()
    return [ output['script'] for output in doge.fetchtx(txn)['outputs'] if 'OP_RETURN' in output['script']][0][10:]

In [ ]:
opreturns=[bytes.fromhex(tx2opreturn(txn)) for txn in lyric_txs]
opreturns

In [ ]:
import pandas as pd
opreturns_series=pd.Series(opreturns)
opreturns_series.drop_duplicates(inplace=True)
opreturns=opreturns_series.to_list()
opreturns

In [ ]:
[ opreturn[4:] for opreturn in opreturns] 

In [ ]:
[bytes.fromhex(tx2opreturn(txn)[8:]).decode() for txn in lyric_txs]

In [ ]:
print(''.join([bytes.fromhex(tx2opreturn(txn)[4:]).decode() for txn in lyric_txs]))

In [ ]:
def extract_multiline(txs):
    import pandas as pd
    byte_returns=[bytes.fromhex(tx2opreturn(txn)) for txn in txs]
    unique_byte_returns=sorted(pd.Series(byte_returns).drop_duplicates().to_list())
    return ''.join([elem[4:].decode() for elem in unique_byte_returns])

In [ ]:
joy_div=extract_multiline(lyric_txs)
print(joy_div)

In [ ]:
spineta=extract_multiline(lyric_txs1)
print(spineta)

In [ ]:
#weird opreturn i found in dogecoin blockchain
bytes.fromhex('737761703a424e422e425553442d4244313a626e6231726e7667326b3537776e6d32747578336c386e34647672753671676465746e6b7739647368343a313738333134373632343939')